In [4]:
from time import perf_counter
import numpy as np,os
import pandas as pd
from collections import defaultdict
from sklearn.utils import shuffle
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from imblearn.under_sampling import InstanceHardnessThreshold, RandomUnderSampler, EditedNearestNeighbours
from imblearn.pipeline import Pipeline

from sklearn.metrics import matthews_corrcoef, precision_recall_curve, auc, accuracy_score, precision_score, recall_score
import seaborn as sns,matplotlib.pyplot as plt

from library.configs import IMBS, CLFS, ENSEMBLES, CV, SCORERS
from library.utils import evaluate, read_data
from library.cleaners import kDN, ih_prob,FilteringEstimator

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
DATASETS = ['groovy-1_5_7.csv','jruby-1.4.0.csv','lucene-2.9.0.csv','jruby-1.7.0.preview1.csv','groovy-1_6_BETA_1.csv',
        'derby-10.2.1.6.csv','wicket-1.5.3.csv','camel-2.9.0.csv','camel-1.4.0.csv','activemq-5.8.0.csv']
DATASETS = [f for f in os.listdir("JIRA/") if 'csv' in f]
len(DATASETS)

32

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin, clone
class Spyder(BaseEstimator, ClassifierMixin):
    def __init__(self,estimator,K=5):
        self.estimator = estimator
        self.K = K
        
    def sample(self,X,Y,sample_weight):
        # SPIDER2, relabel=False and ampl=weak by default
        # First step, Remove unsafe samples from majority, 
        Xs,Ys = EditedNearestNeighbours(sampling_strategy='majority',n_neighbors=self.K,
                                kind_sel='mode').fit_resample(X,Y)
        
        #Second, upsample unsafe minority samples
        disagreement = kDN(Xs,Ys,K=self.K,weight='uniform')
        unsafe = disagreement>(self.K/2) 
        unsafe[Ys==0] = 0  # Make all from majority-class safe
        print(unsafe.sum(),(Ys==1).sum())
        
        return Xt,Yt,sample_weight
    
    def fit(self, X, Y,sample_weight=None):
        Xf,Yf,sample_weight = self.sample(X, Y, sample_weight)
#         a,b = np.unique(Y,return_counts=True)[1],np.unique(Yf,return_counts=True)[1]
#         print(a.max()/a.min(),b.max()/b.min(),a,b,Xf.shape,len(Xf)/len(X))
        assert len(np.unique(Yf))==2,"Pos class completely filtered out"
        try:
            self.estimator = self.estimator.fit(Xf, Yf,sample_weight=sample_weight)
        except TypeError as e:
            self.estimator = self.estimator.fit(Xf, Yf)
        return self

    @property
    def classes_(self):
        return self.estimator.classes_

    def predict(self, X):
        return self.estimator.predict(X)
    
    def predict_proba(self, X):
        return self.estimator.predict_proba(X) 

In [19]:
X,yn,yr = read_data(DATASETS[5],True)

jruby-1.4.0.csv noise:0.190, imb:3.890,200,778, Shape:(978, 65)


In [27]:
Xs,Ys = EditedNearestNeighbours(sampling_strategy='majority',n_neighbors=5,
                        kind_sel='mode').fit_resample(X,yn)
print(np.bincount(Ys))
#Second, upsample unsafe minority samples
disagreement = kDN(Xs,Ys,K=5,weight='uniform') * 5
unsafe = disagreement>(5/2)
unsafe[Ys==0] = 0  # Make all from majority-class safe

opposite = disagreement[unsafe]
same = 
print(unsafe.sum(),(Ys==1).sum())

[723 200]
95 200


In [29]:
disagreement[unsafe]

array([4., 3., 3., 5., 3., 3., 4., 4., 3., 4., 4., 5., 5., 4., 3., 4., 5.,
       4., 4., 5., 5., 4., 4., 3., 5., 3., 4., 5., 4., 5., 5., 4., 3., 3.,
       5., 3., 4., 4., 5., 4., 3., 4., 4., 5., 4., 5., 5., 4., 3., 3., 4.,
       5., 5., 5., 5., 3., 3., 4., 5., 5., 4., 5., 4., 3., 4., 5., 3., 4.,
       4., 5., 3., 4., 4., 5., 5., 4., 5., 5., 4., 4., 3., 3., 4., 4., 4.,
       4., 5., 3., 5., 5., 5., 3., 3., 4., 3.])

In [24]:
unsafe.sum()

0